# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [3]:
infile = f"../data/leopoldII/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [4]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [5]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [6]:
len(bigram_phrases.vocab.keys())

2604605

Prenons une clé au hasard :

In [7]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

st_-


Le dictionnaire indique le score de cette coocurrence :

In [8]:
bigram_phrases.vocab[key_]

3324

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [9]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [10]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [11]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [12]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [13]:
print(corpus[:100])

[['il', '1', '-', '.'], ['-', '#', '#', '#', 'r', "'", 'i', "'", '"\'"', '*', 'i', 'i', 'theatre', 'ou', 'a_proximite', '.'], ['surement', ',', 'il', 'eut_ete', 'compromis', 'un', 'des', 'premiers', ',', 'car', 'il_etait', 'ardent', 'mazziniste', '.'], ['et', 'dire_que', ',', 's', "'", 'il_avait', 'possede_un', 'simple', 'rat', 'de', 'cave', ',', 'toute_sa', 'destinee', 'etait', 'changee', ',', 'et', 'peut', '-', 'etre', 'aussi', 'celle', 'de', "l_'", 'italie', 'i', 'chronique_locale', '.'], ['--', '--', 'arlon', '.'], ['--', 'exposition', 'des_eglises', 'pauvres', '.'], ['--', 'c', "'", 'est', 'demain', 'que', 's', "'", 'ouvrira', ',', 'rue', 'st', '-', 'jean', ',', "l_'", 'exposition', 'des', 'objets', 'confectionnes', 'par', 'les', 'dames', 'de', "l_'", 'association', 'des_eglises', 'pauvres', ':', 'elle', 'sera', 'egalement', 'accessible', 'au_public', 'le', 'lundi', '5', '.'], ['les', 'deux_jours', "l_'", 'exposition', 'sera_ouverte', 'de', '2', 'a', '6', 'h', '.', 'du_soir', '.']

## Entrainement d'un modèle Word2Vec sur ce corpus

In [14]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: user 1min 12s, sys: 437 ms, total: 1min 12s
Wall time: 30.5 s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [15]:
outfile = f"../data/leopoldII/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [48]:
model = Word2Vec.load("../data/leopoldII/newspapers.model")

### Imprimer le vecteur d'un terme

In [49]:
model.wv["ministre"]

array([-0.31214097, -1.852512  ,  0.71405685,  0.5509615 , -2.4458945 ,
        0.93208545,  0.85143256,  1.3133869 , -2.9048648 ,  0.4518651 ,
        1.6864238 ,  1.141707  , -0.9972724 , -0.18815526, -1.0679187 ,
       -1.3020313 ,  0.6147622 ,  2.3466246 , -0.02992805, -1.180652  ,
        1.9188246 ,  2.0818365 ,  1.3316755 , -1.4174677 , -2.2411113 ,
        1.5436935 ,  0.18232949, -1.2818879 ,  2.2229252 , -1.7415988 ,
       -4.0903354 , -1.7315545 ], dtype=float32)

### Calculer la similarité entre deux termes

In [50]:
model.wv.similarity("congo", "exportation")

0.21691632

### Chercher les mots les plus proches d'un terme donné

In [51]:
model.wv.most_similar("exportation", topn=10)

[('emission', 0.9178957343101501),
 ('augmentation', 0.8942883610725403),
 ('et_la_speculation', 0.8897349238395691),
 ('action', 0.8486761450767517),
 ('importation', 0.848074197769165),
 ('amortissement', 0.8475116491317749),
 ('alcool', 0.8471620678901672),
 ('exploitation', 0.8441718816757202),
 ('ecole_francaise', 0.8368322849273682),
 ('accumulation', 0.834017813205719)]

In [52]:
model.wv.most_similar("exposition", topn=10)

[('inauguration', 0.8244574666023254),
 ('adjudication', 0.7931884527206421),
 ('ouverture', 0.7885424494743347),
 ('ecole', 0.7780244946479797),
 ('au_siege', 0.7703685164451599),
 ('la_societe', 0.7593361139297485),
 ('express', 0.7487666606903076),
 ('eglise', 0.7452453970909119),
 ('hotel', 0.7401634454727173),
 ('assemblee_generale', 0.7386838793754578)]

In [53]:
model.wv.most_similar("art", topn=10)

[('achat', 0.7775892615318298),
 ('agrement', 0.7743470668792725),
 ('infanterie', 0.768336296081543),
 ('une_collection', 0.7636668682098389),
 ('ameublement', 0.7594934105873108),
 ('ecole', 0.7557386159896851),
 ('),_executees', 0.7507432699203491),
 ('echappement', 0.7382581830024719),
 ('auberge', 0.7336875796318054),
 ('epuisement', 0.7329124212265015)]

In [54]:
model.wv.most_similar("art_decoratif", topn=10)

[('ethnographie', 0.8825387954711914),
 ('ecauss', 0.8753301501274109),
 ('humanites_modernes_ou', 0.8701314330101013),
 ('occident', 0.8673361539840698),
 ('anglenr', 0.8586602210998535),
 ('outremer_,,_div', 0.8572244644165039),
 ('arr_.)', 0.8518468141555786),
 ('outremer_cap', 0.8499533534049988),
 ('eleet_.)', 0.849478542804718),
 ('obourg', 0.847644567489624)]

In [55]:
model.wv.most_similar("colonisation", topn=10)

[('responsabilite', 0.9362757802009583),
 ('religion', 0.93475741147995),
 ('solidarite', 0.9323123097419739),
 ('sagesse', 0.9280848503112793),
 ('polilique', 0.9274063110351562),
 ('la_bourgeoisie', 0.9262202382087708),
 ('collaboration', 0.92580246925354),
 ('promesse', 0.9256329536437988),
 ('souverainete', 0.9249749779701233),
 ('consideration', 0.9242480993270874)]

In [56]:
model.wv.most_similar("indigene", topn=10)

[('reglementaire', 0.8267008066177368),
 ('comprenait', 0.8248342275619507),
 ('dato', 0.8183184862136841),
 ('assembleo_generale', 0.8107446432113647),
 ('emportant', 0.8100875020027161),
 ('ecoule', 0.8075682520866394),
 ('cette_annee', 0.8066053986549377),
 ('effectue', 0.8043825626373291),
 ('tous_les_ans', 0.8022083044052124),
 ('delivre', 0.8012433648109436)]

In [57]:
model.wv.most_similar("congolais", topn=10)

[('coloniales', 0.9225746989250183),
 ('necessaires', 0.9091405868530273),
 ('dispositions', 0.8899512887001038),
 ('les_droits', 0.885262131690979),
 ('du_pays', 0.8811383247375488),
 ('reformes', 0.880110502243042),
 ('des_indigenes', 0.8799312710762024),
 ('parlementaires', 0.8789421319961548),
 ('des_partis', 0.8756430149078369),
 ('religieux', 0.8752180337905884)]

In [59]:
model.wv.most_similar("ivoire", topn=10)

[('elegance', 0.9036751389503479),
 ('etoffe', 0.8918654918670654),
 ('alcool', 0.8820372223854065),
 ('etain', 0.8793249130249023),
 ('oppression', 0.8789166808128357),
 ('elevage', 0.8758142590522766),
 ('ampleur', 0.8753540515899658),
 ('espace', 0.872124195098877),
 ('accumulation', 0.8662607669830322),
 ('entretien', 0.8654978275299072)]

In [60]:
model.wv.most_similar("palais", topn=10)

[('grand_hotel', 0.7860857248306274),
 ('situe', 0.7727888226509094),
 ('en_face', 0.7692857980728149),
 ('usines_cambier', 0.7573888301849365),
 ('.--_salons', 0.7481986284255981),
 ('la_gare', 0.7406401634216309),
 ('hotel', 0.7364373207092285),
 ('la_station', 0.721713125705719),
 ('depart', 0.7174635529518127),
 ('proximite', 0.7110736966133118)]

In [61]:
model.wv.most_similar("exotique", topn=10)

[('las_palmas', 0.9319145679473877),
 ('telephona', 0.9262458086013794),
 ('1651', 0.9211452007293701),
 ('sports_courses', 0.9190716743469238),
 ('kaz', 0.9170840382575989),
 ('18_professeurs', 0.9136096239089966),
 ('mayor', 0.9116635918617249),
 ('("', 0.9102340936660767),
 ('sera_remise', 0.9089506268501282),
 ('thermales_et_hivernales', 0.9068217277526855)]

In [62]:
model.wv.most_similar("colonie", topn=10)

[('presse', 0.9302968382835388),
 ('civilisation', 0.9257255792617798),
 ('conscience', 0.9253501296043396),
 ('matiere', 0.9248175621032715),
 ('prosperite', 0.9231202006340027),
 ('consideration', 0.9229942560195923),
 ('responsabilite', 0.9223616123199463),
 ('faute', 0.921610951423645),
 ('solution', 0.9209235310554504),
 ('liberte', 0.9205229878425598)]

### Faire des recherches complexes à travers l'espace vectoriel

In [63]:
print(model.wv.most_similar(positive=['paris', 'londres'], negative=['belgique']))

[('change', 0.7094481587409973), ('vienne', 0.7000303864479065), ('new_-_york', 0.6647735834121704), ('saint_-_petersbourg', 0.6473316550254822), ('berlin', 0.6448163390159607), ('amsterdam', 0.6252720952033997), ('constantinople', 0.6161597371101379), ('le_soir', 0.6114900708198547), ('hambourg', 0.6111112833023071), ('paria', 0.6052391529083252)]
